## Pipeline de Processamento e Armazenamento de Embeddings para Ementas

Este pipeline foi desenvolvido para **processar arquivos PDF de ementas de disciplinas**, com o objetivo de **gerar representações vetoriais (embeddings)** e armazená-las em um banco vetorial para consultas futuras.

### Ferramentas Principais

* **Manipulação de PDFs**: `PyMuPDF` (fitz) para extração de texto.
* **Processamento de texto**: Bibliotecas Python padrão (`re`, `os`, etc.) para limpeza e manipulação.
* **Geração de Embeddings**: [sentence-transformers](https://www.sbert.net/) para a criação de vetores semânticos.
* **Armazenamento Vetorial**: [ChromaDB](https://www.trychroma.com/) para gerenciar e indexar os embeddings.

### Estratégia de Chunking e Metadata

Cada PDF de ementa é cuidadosamente processado e dividido em **chunks semânticos**, seguindo a estrutura padrão das ementas. Isso inclui seções como:

* Identificação (código, nome da disciplina, carga horária)
* Objetivo da disciplina
* Ementa (resumo do conteúdo)
* Conteúdo Programático (dividido por unidades)
* Bibliografia Básica
* Bibliografia Complementar

Cada chunk é enriquecido com **metadados** relevantes, como o código da disciplina, o tipo de seção e a fonte (nome do arquivo PDF), para facilitar futuras buscas e filtragens.

### Modelos de Embeddings Utilizados

Para cada chunk, embeddings são gerados usando uma seleção de modelos da família SentenceTransformer, garantindo versatilidade e a possibilidade de comparação de desempenho:

* `all-MiniLM-L6-v2`
* `all-mpnet-base-v2`
* `paraphrase-multilingual-MiniLM-L12-v2`
* `distiluse-base-multilingual-cased-v2`
* `stsb-xlm-r-multilingual`
* `neuralmind/bert-base-portuguese-cased`

### Armazenamento no ChromaDB

Os **embeddings, os textos originais e seus metadados** são armazenados em **coleções separadas no ChromaDB**, uma para cada modelo de embedding. Essa abordagem oferece:

* **Comparação de performance**: Facilita a avaliação do desempenho de diferentes modelos em cenários de consulta.
* **Flexibilidade na escolha do modelo**: Permite selecionar o modelo mais adequado para cada tipo de consulta.
* **Rastreabilidade**: Os metadados garantem a origem e o contexto de cada chunk.

### Fluxo do Pipeline

1.  **Extração**: Leitura e extração do texto dos PDFs de ementas.
2.  **Pré-processamento**: Limpeza e tratamento do texto extraído.
3.  **Chunking**: Geração de chunks semânticos baseados na estrutura da ementa.
4.  **Embeddings**: Criação de vetores para cada chunk usando todos os modelos definidos.
5.  **Armazenamento**: Persistência dos embeddings, textos e metadados nas coleções específicas do ChromaDB.

In [ ]:
# Célula 1: Importando bibliotecas
import os
import re
from typing import List, Dict, Tuple
from dotenv import load_dotenv
import torch
from sentence_transformers import SentenceTransformer
import chromadb
import fitz  # PyMuPDF

import numpy as np
from pathlib import Path
import pandas as pd

In [ ]:
# Célula 2: Configuração dos Modelos de Embeddings
def setup_embedding_models() -> List[Tuple[str, SentenceTransformer]]:
    models = [
        ("all-MiniLM-L6-v2", SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')),
        ("all-mpnet-base-v2", SentenceTransformer('sentence-transformers/all-mpnet-base-v2')),
        ("paraphrase-multilingual-MiniLM-L12-v2", SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')),
        ("distiluse-base-multilingual-cased-v2", SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')),
        ("stsb-xlm-r-multilingual", SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')),
        ("neuralmind-bert-base-portuguese-cased", SentenceTransformer('neuralmind/bert-base-portuguese-cased'))
    ]
    return models

In [5]:
# Célula 3: Configuração do ChromaDB (adaptada para ementas)
def setup_chromadb(base_dir: str, models: List[Tuple[str, SentenceTransformer]]) -> Tuple[chromadb.PersistentClient, List[chromadb.Collection]]:
    chroma_dir = Path(base_dir) / "chroma_db"
    chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

    collections = []
    existing_collections = [col.name for col in chroma_client.list_collections()]

    for model_name, _ in models:
        collection_name = f"ementas-disciplinas_{model_name}"

        if collection_name in existing_collections:
            print(f"Excluindo coleção existente: {collection_name}")
            chroma_client.delete_collection(name=collection_name)

        print(f"Criando nova coleção: {collection_name}")
        collection = chroma_client.create_collection(
            name=collection_name,
            metadata={"description": f"Ementas de disciplinas usando modelo {model_name}"}
        )
        collections.append(collection)

    return chroma_client, collections

In [ ]:
# Célula 4: Funções utilitárias para extração e limpeza

def extract_text_from_pdf(pdf_path: str) -> str:
    text = ""
    try:
        pdf_document = fitz.open(pdf_path)
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text += page.get_text("text", sort=True)
        pdf_document.close()
    except Exception as e:
        print(f"Erro ao extrair texto do PDF: {e}")
        return None
    return text

def clean_chunk_text(text: str, chunk_type: str = None) -> str:
    if chunk_type == 'identificacao':
        # Extrair código da disciplina
        codigo_match = re.search(r'(CC|PP|CH|OP|MEIU)\d+[A-Z0-9]', text)
        codigo = codigo_match.group(0) if codigo_match else "N/A"

        # Extrair nome da disciplina - Método mais robusto
        disciplina = "N/A"
        if codigo_match:
            # Tentar diferentes padrões para encontrar o nome da disciplina
            patterns = [
                # Padrão 1: Após o código até Nota/Conceito
                rf'{codigo}\s+(.*?)(?=Nota/Conceito|Frequência|Presencial|Semestral)',
                # Padrão 2: Entre "Disciplina/Unidade" e "Nota/Conceito"
                r'Disciplina/Unidade[^>]*?([^>]*?)(?=Nota/Conceito|Frequência|Presencial|Semestral)',
                # Padrão 3: Após "Curricular" até "Nota/Conceito"
                r'Curricular\s*([^>]*?)(?=Nota/Conceito|Frequência|Presencial|Semestral)'
            ]

            for pattern in patterns:
                disciplina_match = re.search(pattern, text, re.DOTALL)
                if disciplina_match:
                    disciplina = disciplina_match.group(1).strip()
                    # Limpar o nome da disciplina
                    disciplina = re.sub(r'\s+', ' ', disciplina)  # Remove múltiplos espaços
                    disciplina = re.sub(r'^\s*Curricular\s*', '', disciplina)  # Remove "Curricular" do início
                    disciplina = disciplina.strip()
                    if disciplina and disciplina != "N/A":
                        break

        # Extrair carga horária total
        carga_total_match = re.search(r'Total\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+(\d+)', text)
        if not carga_total_match:
            # Tentar padrões alternativos
            carga_patterns = [
                r'Total\s+(\d+)\s+AT:',
                r'Total\s*:\s*(\d+)',
                r'Total\s+(\d+)\s*h',
                r'Total\s+(\d+)'
            ]
            for pattern in carga_patterns:
                carga_match = re.search(pattern, text)
                if carga_match:
                    carga_total_match = carga_match
                    break

        carga_horaria = carga_total_match.group(1) + " Horas" if carga_total_match else "N/A"

        # Se ainda não encontrou a disciplina, tentar extrair do metadata
        if disciplina == "N/A" or not disciplina:
            disciplina_match = re.search(r'Disciplina:\s*([^>]*?)(?=\.|$)', text, re.DOTALL)
            if disciplina_match:
                disciplina = disciplina_match.group(1).strip()

        # Formatar o texto final
        text = f"Informações da disciplina {codigo} {disciplina}. Carga Horária {carga_horaria}"
        return text

    # Para os outros chunks, limpeza padrão
    lines = text.split('\n')
    lines = [line for line in lines if line.strip() and line.strip().lower() not in [
        'ordem', 'ementa', 'conteúdo', 'conteudo', 'resumo da alteração', '#',
        'ministério da educação', 'universidade tecnológica federal do paraná', 'campus medianeira',
        'informações da disciplina', 'por conteúdo', 'modalidade', 'código', 'disciplina/unidade',
        'da oferta', 'ofertado', 'curricular', 'modo de avaliação', 'disciplina', 'at', 'ap', 'aps',
        'anp', 'apcc', 'chead', 'che', 'total'
    ]]
    text = '\n'.join(lines)
    text = re.sub(r'^\d+\s+', '', text)
    text = re.sub(r'(Bibliografia Complementar\s*)+$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_section_from_pdf(text: str, section_start: str, section_end: str = None) -> str:
    if section_end:
        pattern = f"{section_start}(.*?){section_end}"
    else:
        pattern = f"{section_start}(.*?)(?=(?:Bibliografia|Resumo da Alteração|$))"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

def extract_metadata_from_header(text: str) -> dict:
    metadata = {}
    # Extrair código e nome da disciplina de forma mais robusta
    codigo_match = re.search(r'(CC|PP)\d+[A-Z]', text)
    if codigo_match:
        metadata['codigo'] = codigo_match.group(0)
        # Tentar encontrar o nome da disciplina após o código
        disciplina_text = text[codigo_match.end():].split('\n')[0]
        metadata['disciplina'] = disciplina_text.strip()
    # Extrair modalidade e oferta
    metadata['modalidade'] = 'Presencial' if 'Presencial' in text else 'Não Presencial'
    metadata['oferta'] = 'Semestral' if 'Semestral' in text else 'Anual'
    return metadata

In [7]:
# Célula 5: Processamento dos PDFs e chunking estruturado

def process_pdf_content(text: str, filename: str) -> tuple:
    """
    Processa o conteúdo do PDF e retorna metadados e chunks.
    """
    metadata = extract_metadata_from_header(text)

    # Extrair código do nome do arquivo se não foi encontrado no texto
    if not metadata.get('codigo'):
        metadata['codigo'] = os.path.splitext(filename)[0]  # Remove a extensão .pdf

    # Tentar extrair disciplina do texto usando o código
    if not metadata.get('disciplina'):
        disciplina_pattern = f"{metadata['codigo']}(.*?)(?=Frequência|Presencial|Nota|$)"
        disciplina_match = re.search(disciplina_pattern, text, re.DOTALL)
        if disciplina_match:
            disciplina = disciplina_match.group(1).strip()
            metadata['disciplina'] = disciplina
        else:
            # Se ainda não encontrou, procurar após "Disciplina" ou "Unidade Curricular"
            disc_match = re.search(r'(?:Disciplina|Unidade Curricular)[:\s]+(.*?)(?=Frequência|Presencial|Nota|$)', text, re.DOTALL)
            if disc_match:
                metadata['disciplina'] = disc_match.group(1).strip()
            else:
                metadata['disciplina'] = f"Disciplina {metadata['codigo']}"

    chunks = []

    # Chunk de identificação
    chunks.append({
        'text': clean_chunk_text(text[:500], chunk_type='identificacao'),
        'tipo': 'identificacao',
        'metadata': metadata.copy()
    })

    # Chunk de objetivo
    objetivo = extract_section_from_pdf(text, "Objetivo", "Ementa")
    if objetivo:
        chunks.append({
            'text': clean_chunk_text(objetivo),
            'tipo': 'objetivo',
            'metadata': metadata.copy()
        })

    # Chunk de ementa
    ementa = extract_section_from_pdf(text, "Ementa", "Conteúdo Programático")
    if ementa:
        chunks.append({
            'text': clean_chunk_text(ementa),
            'tipo': 'ementa',
            'metadata': metadata.copy()
        })

    # Chunks de conteúdo programático
    conteudo = extract_section_from_pdf(text, "Conteúdo Programático", "Bibliografia")
    if conteudo:
        topicos = re.findall(r'(\d+\s*.*?(?=\d+\s*|Bibliografia|$))', conteudo, re.DOTALL)
        for topico in topicos:
            chunks.append({
                'text': clean_chunk_text(topico.strip()),
                'tipo': 'conteudo_programatico',
                'metadata': {**metadata.copy(), 'secao': 'Tópico ' + topico.split()[0]}
            })

    # Chunks de bibliografia
    bibliografia_basica = extract_section_from_pdf(text, "Bibliografia Básica", "Bibliografia Complementar")
    if bibliografia_basica:
        chunks.append({
            'text': clean_chunk_text(bibliografia_basica),
            'tipo': 'bibliografia_basica',
            'metadata': metadata.copy()
        })

    bibliografia_complementar = extract_section_from_pdf(text, "Bibliografia Complementar", "Resumo")
    if bibliografia_complementar:
        chunks.append({
            'text': clean_chunk_text(bibliografia_complementar),
            'tipo': 'bibliografia_complementar',
            'metadata': metadata.copy()
        })

    return metadata, chunks

In [8]:
# Célula 6: Processamento e inserção em múltiplas coleções
def process_pdf_and_add_to_collections(
    pdf_path: str,
    models: List[Tuple[str, SentenceTransformer]],
    collections: List[chromadb.Collection]
):
    """
    Processa um único PDF e adiciona seus chunks em todas as coleções.
    """
    try:
        # Extrair nome do arquivo e texto
        source = os.path.basename(pdf_path)
        text = extract_text_from_pdf(pdf_path)

        if not text:
            print(f"Não foi possível extrair texto de {pdf_path}. Pulando...")
            return

        # Processar conteúdo e gerar chunks
        metadata, chunks = process_pdf_content(text, source)

        # Adicionar informações de fonte
        metadata['source'] = source
        metadata['codigo'] = os.path.splitext(source)[0]

        print(f"\nProcessando: {source}")
        print(f"Disciplina: {metadata.get('disciplina', 'N/A')}")
        print(f"Código: {metadata['codigo']}")

        # Para cada modelo/coleção, gerar embeddings e inserir
        for (model_name, embedding_model), collection in zip(models, collections):
            print(f"  Inserindo na coleção: {collection.name}")

            for i, chunk in enumerate(chunks):
                chunk_text = chunk['text']
                chunk_metadata = {
                    **chunk['metadata'],
                    'source': source,
                    'tipo': chunk['tipo'],
                    'codigo': metadata['codigo']
                }

                # Gerar embedding específico para este modelo
                embedding = embedding_model.encode(chunk_text).tolist()

                # ID único para este chunk nesta coleção
                chunk_id = f"{metadata['codigo']}_{chunk['tipo']}_{i}_{model_name}"

                # Adicionar à coleção
                collection.add(
                    embeddings=[embedding],
                    documents=[chunk_text],
                    metadatas=[chunk_metadata],
                    ids=[chunk_id]
                )

            print(f"    {len(chunks)} chunks adicionados.")

    except Exception as e:
        print(f"Erro ao processar {pdf_path}: {str(e)}")
        return None

def process_all_pdfs_in_directory(
    pdf_dir: str,
    models: List[Tuple[str, SentenceTransformer]],
    collections: List[chromadb.Collection]
) -> Dict:
    """
    Processa todos os PDFs em um diretório e adiciona em todas as coleções.
    Retorna um dicionário com os chunks organizados por disciplina.
    """
    # Verificar e listar arquivos PDF
    if not os.path.exists(pdf_dir):
        print(f"Diretório não encontrado: {pdf_dir}")
        return {}

    pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir)
                 if f.lower().endswith('.pdf')]

    if not pdf_files:
        print("Nenhum arquivo PDF encontrado.")
        return {}

    print(f"Encontrados {len(pdf_files)} arquivos PDF em {pdf_dir}")

    # Dicionário para armazenar chunks por disciplina
    chunks_por_disciplina = {}

    # Processar cada PDF
    for pdf_file in pdf_files:
        try:
            # Extrair texto e processar chunks
            source = os.path.basename(pdf_file)
            text = extract_text_from_pdf(pdf_file)

            if not text:
                continue

            # Processar conteúdo e gerar chunks
            metadata, chunks = process_pdf_content(text, source)
            codigo = metadata['codigo']

            # Armazenar chunks no dicionário
            chunks_por_disciplina[codigo] = chunks

            print(f"\nProcessando: {source}")
            print(f"Disciplina: {metadata.get('disciplina', 'N/A')}")
            print(f"Código: {codigo}")

            # Adicionar em todas as coleções
            for (model_name, embedding_model), collection in zip(models, collections):
                print(f"  Inserindo na coleção: {collection.name}")

                for i, chunk in enumerate(chunks):
                    chunk_text = chunk['text']
                    chunk_metadata = {
                        **chunk['metadata'],
                        'source': source,
                        'tipo': chunk['tipo'],
                        'codigo': codigo
                    }

                    embedding = embedding_model.encode(chunk_text).tolist()
                    chunk_id = f"{codigo}_{chunk['tipo']}_{i}_{model_name}"

                    collection.add(
                        embeddings=[embedding],
                        documents=[chunk_text],
                        metadatas=[chunk_metadata],
                        ids=[chunk_id]
                    )

                print(f"    {len(chunks)} chunks adicionados.")

        except Exception as e:
            print(f"Erro ao processar {pdf_file}: {str(e)}")
            continue

    print("\nProcessamento concluído!")
    return chunks_por_disciplina

In [ ]:
# Célula 7: Execução do Pipeline
if __name__ == "__main__":
    # Configurar modelos de embedding
    models = setup_embedding_models()

    # Configurar ChromaDB e coleções
    base_dir = os.getcwd()
    chroma_client, collections = setup_chromadb(base_dir, models)

    # Configurar diretório dos PDFs
    pdf_dir = os.path.normpath(os.path.join(
        base_dir, "..", "Webscraping", "Files", "PDF", "Ementas", "Ciência da Computação"
    ))

    # Processar todos os arquivos PDF
    chunks_por_disciplina = process_all_pdfs_in_directory(pdf_dir, models, collections)

    # Imprimir resumo do processamento
    print("\nResumo do processamento:")
    print(f"Total de disciplinas processadas: {len(chunks_por_disciplina) if chunks_por_disciplina else 0}")

In [ ]:
# Célula de Visualização: Exibe chunks principais de todas as disciplinas
import os
from typing import List, Dict, Tuple
from sentence_transformers import SentenceTransformer
import re
from PyPDF2 import PdfReader

# (Reutilize as funções extract_text_from_pdf, clean_chunk_text, extract_section_from_pdf, extract_metadata_from_header, process_pdf_content da Célula 4)

def display_chunks_overview(pdf_dir: str):
    """
    Exibe os chunks de identificação, objetivo e ementa de cada PDF no diretório.
    """
    # Listar arquivos PDF
    pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print("Nenhum arquivo PDF encontrado.")
        return

    print(f"Encontrados {len(pdf_files)} arquivos PDF em {pdf_dir}")

    # Processar cada PDF
    for pdf_file in pdf_files:
        try:
            # Extrair nome do arquivo
            source = os.path.basename(pdf_file)

            # Extrair texto do PDF
            text = extract_text_from_pdf(pdf_file)
            if not text:
                print(f"Não foi possível extrair texto de {pdf_file}. Pulando...")
                continue

            # Processar conteúdo e gerar chunks
            metadata, chunks = process_pdf_content(text, source)

            print(f"\nArquivo: {source}")
            print(f"Disciplina: {metadata.get('disciplina', 'N/A')}")
            print(f"Código: {metadata['codigo']}")
            print("=" * 40)

            # Exibir chunks principais
            for chunk in chunks:
                if chunk['tipo'] in ['identificacao', 'objetivo', 'ementa']:
                    print(f"\nTipo: {chunk['tipo']}")
                    print(f"Texto: {chunk['text']}")
                    print("-" * 20)

        except Exception as e:
            print(f"Erro ao processar {pdf_file}: {str(e)}")

# Exemplo de uso:
if __name__ == "__main__":
    base_dir = os.getcwd()
    pdf_dir = os.path.normpath(os.path.join(
        base_dir, "..", "Webscraping", "Files", "PDF", "Ementas", "Ciência da Computação"
    ))
    display_chunks_overview(pdf_dir)

In [10]:
# Célula: Visualizar todos os chunks e metadados da disciplina do documento OP63P.pdf

# Defina o caminho do PDF
base_dir = os.getcwd()
pdf_path = os.path.normpath(os.path.join(
    base_dir, "..", "Webscraping", "Files", "PDF", "Ementas", "Ciência da Computação", "CC56E.pdf"
))

# Extraia o texto do PDF
text = extract_text_from_pdf(pdf_path)
if not text:
    print("Não foi possível extrair texto do PDF.")
else:
    # Gere os chunks e metadados
    metadata, chunks = process_pdf_content(text, "OP63P.pdf")
    print(f"Arquivo: OP63P.pdf")
    print(f"Disciplina: {metadata.get('disciplina', 'N/A')}")
    print(f"Código: {metadata.get('codigo', 'N/A')}")
    print("=" * 60)
    for i, chunk in enumerate(chunks):
        print(f"\nChunk {i+1}:")
        print(f"Tipo: {chunk['tipo']}")
        print(f"Texto: {chunk['text']}")
        print(f"Metadados: {chunk['metadata']}")
        print("-" * 40)

Arquivo: OP63P.pdf
Disciplina: Aspectos Formais Da
Computação
Código: CC56E

Chunk 1:
Tipo: identificacao
Texto: Informações da disciplina CC56E Aspectos Formais Da Computação. Carga Horária 45 Horas
Metadados: {'codigo': 'CC56E', 'disciplina': 'Aspectos Formais Da\nComputação', 'modalidade': 'Presencial', 'oferta': 'Semestral'}
----------------------------------------

Chunk 2:
Tipo: objetivo
Texto: Dar ao aluno noção formal de algoritmo, complexidade e computabilidade e do problema de decisão, de modo a deixá-lo consciente das limitações da ciência da computação. Aparelhá-lo com as ferramentas de modo a habilitá-lo a melhor solucionar problemas com o auxílio do computador e técnicas de computação.
Metadados: {'codigo': 'CC56E', 'disciplina': 'Aspectos Formais Da\nComputação', 'modalidade': 'Presencial', 'oferta': 'Semestral'}
----------------------------------------

Chunk 3:
Tipo: ementa
Texto: Problemas solucionáveis e não solucionáveis. Aplicação da teoria dos grafos. Complexidade